In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.stats import norm
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')
%matplotlib inline

# Loading Data 


In [ ]:
Data = pd.read_csv('../input/data-science-london-scikit-learn/train.csv',header=None)
Label = pd.read_csv('../input/data-science-london-scikit-learn/trainLabels.csv',header=None,names=['label'])


# Get Describtion & Information About Data

In [ ]:
Data.info()
D=Data.copy(deep=True)

In [ ]:
Data.describe()

# Get Correlation Between Label Column And Train Data 

In [ ]:
x=Data.corrwith(Label['label'])
x = np.asarray(x).reshape(40,1)

fig, ax = plt.subplots(figsize=(10, 15))
sns.heatmap(x, annot = True,cmap= 'Blues')#, fmt='.6g'

### Replace Columns That Has Bad Correlation With Label as they Made Miss Leading by a Two New columns has Correlation one Positive And one Negative 

In [ ]:
EC = Data.columns
Drop_col=[]
for i in EC:
    if(((Data[i].corr(Label['label']) < 0.01) and (Data[i].corr(Label['label'])> 0)) or 
    ((Data[i].corr(Label['label']) < 0) and (Data[i].corr(Label['label'])> -0.09))):
        Drop_col.append(i)
        Data.drop(i, axis=1, inplace=True)
        
EC=Data.columns


# Using PCA To Make 2 New Columns By Combination Of Bad Correlation Columns 

In [ ]:
x=D.iloc[:,Drop_col]

model = PCA(n_components= 2)
model.fit(x)

data = model.transform(x)
data = pd.DataFrame(data)
Data = pd.concat([Data, data], axis=1)

In [ ]:
x=Data.corrwith(Label['label'])
x = np.asarray(x).reshape(26,1)

fig, ax = plt.subplots(figsize=(10, 15))
sns.heatmap(x, annot = True,cmap= 'Blues')

In [ ]:
Data.info()

# First Graph Represent That The Maximun number of Label is 1
### Second Graph represent that and specific The number of Unique Values in this Column is 2( 0 , 1 )

In [ ]:
sns.distplot(Label['label'], fit=norm);
fig = plt.figure()
res = stats.probplot(Label['label'], plot=plt)

# Data Visualization
### Get Columns Normal Graph To Make Sure That Data Has Normal Distribution With Density

In [ ]:
horizontal_concat = pd.concat([Data, Label['label']], axis=1)
horizontal_concat=pd.DataFrame(horizontal_concat).astype('object')
sns.pairplot(horizontal_concat, corner=True)

In [ ]:
for i in Data.columns:
    sns.distplot(Data[i], fit=norm);
    fig = plt.figure()

# Standard Scaler for Data

In [ ]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(Data)

In [ ]:
Out=Label['label']
Out=np.array(Out)

# Doing All Previous Data Preprocessing On Test Data

In [ ]:
Test_Data = pd.read_csv('../input/data-science-london-scikit-learn/test.csv',header=None)
Test_Data.info()
D=Test_Data.copy(deep=True)
Test_Data = Test_Data.loc[:,EC]
x=D.iloc[:,Drop_col]
model = PCA(n_components= 2)
model.fit(x)
data = model.transform(x)
data = pd.DataFrame(data)
Test_Data = pd.concat([Test_Data, data], axis=1)

In [ ]:
Test_Data.info()
X_Test = sc_X.fit_transform(Test_Data)

# Machine Learning Modeling


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix



In [ ]:
params=[{'solver':('lbfgs','newton-cg','saga'),'penalty':('l2','l1','elasticnet')},
        {'kernel':('linear', 'rbf'), 'C':[1, 10]},
        {'penalty':('l2','l1','elasticnet'),'multi_class':('ovr', 'crammer_singer')},
        {'max_depth':[3,5,10,15],'n_estimators':[50,100,150,200]},
        {'algorithm':('auto', 'ball_tree', 'kd_tree', 'brute')},
        {'alpha': [0.1,0.0001,1],'penalty':('l2','l1','elasticnet')},
        {'max_depth':[3,5,10,15]}]


LR = LogisticRegression(n_jobs=-1)
SV = SVC()
LS = LinearSVC(random_state=False)
RR = RandomForestClassifier(random_state=False)
KC = KNeighborsClassifier(n_neighbors = 5,weights= 'uniform')
SC = SGDClassifier(shuffle=True,n_jobs=-1,random_state=False)
DC = DecisionTreeClassifier(random_state=False)

models=[LR,SV,LS,RR,KC,SC,DC]
pred=[]

for model,param in zip(models,params):
    print('Model is ',model)
    for i in range (3,11):
        GridSearchModel = GridSearchCV(model,param, cv = i,return_train_score=True, n_jobs=-1)
        GridSearchModel.fit(X_train,Out)
        VD_predict=GridSearchModel.predict(X_Test)       
        VDedict=GridSearchModel.predict(X_train)
        cm = confusion_matrix(Out, VDedict)
        pred.append((GridSearchModel.best_score_,cm,[VD_predict]))
        sorted(GridSearchModel.cv_results_.keys())
        print('CV = ',i)
        print('Best Score is :', GridSearchModel.best_score_)
        print('Best Parameters are :', GridSearchModel.best_params_)
        print('confusion matrix = ')
        print(cm)
        print('-------------------------------')
    print('\t\t*****************************************************************')



# The Best Score is 90 %

In [ ]:
pred.sort(key=lambda y: y[0])
sns.heatmap(pred[-1][1], center = True)
plt.show()
prediction=pred[-1][-1]
prediction=pd.DataFrame(prediction)
prediction=prediction.iloc[0,:]


In [ ]:
output = prediction
output.to_csv('Submission.csv', index=False)

In [ ]:
output